In [2]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\abhin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [14]:
import json
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import findspark

import pyodbc

# Connection details for Azure SQL Database
server = 'arthasqldatabasedemo.database.windows.net'
database = 'arthachicagodemo'
username = 'sqladmin'
password = 'q8sisplVar6madeg'
driver= '{ODBC Driver 17 for SQL Server}'

# Create a connection
connection_string = f'DRIVER={driver};SERVER={server};PORT=1433;DATABASE={database};UID={username};PWD={password}'

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()


os.environ['PYSPARK_PYTHON'] = 'C:\\Users\\abhin\\AppData\\Local\\Programs\\Python\\Python38\\python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:\\Users\\abhin\\AppData\\Local\\Programs\\Python\\Python38\\python.exe'


findspark.init(r'C:\spark\spark-3.4.3-bin-hadoop3')
# Step 1: Set up the Spark Session
spark = SparkSession.builder \
    .appName("YouTube Analytics to PySpark") \
    .getOrCreate()

# Step 2: Authentication and Setting Up YouTube Analytics API Client
SCOPES = ["https://www.googleapis.com/auth/yt-analytics.readonly"]
CLIENT_SECRETS_FILE = 'credentials.json'

# Function to authenticate and get YouTube Analytics service
def get_authenticated_service():
    creds = None
    # The token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=8080)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    youtube_analytics = build('youtubeAnalytics', 'v2', credentials=creds)
    return youtube_analytics

# Step 3: Fetch YouTube Analytics Data
def fetch_youtube_data():
    youtube_analytics = get_authenticated_service()
    
    # Replace with your YouTube channel ID
    channel_id = "MINE" # use 'MINE' to get the data for your authenticated channel

    # Make an API request to fetch analytics data (e.g., views, subscribers)
    response = youtube_analytics.reports().query(
        ids='channel=={}'.format(channel_id),
        startDate='2023-01-01', # Adjust the start date based on your needs
        endDate='2024-01-01', # Adjust the end date based on your needs
        metrics='views,estimatedMinutesWatched,subscribersGained,subscribersLost,likes,comments',
        dimensions='day',
        sort='day',
        maxResults=1000
    ).execute()
    
    # Print raw response to understand structure
    #print(json.dumps(response, indent=4))
    
    # Convert the response to a list of dictionaries (for PySpark processing)
    data = response.get('rows', [])
    return data

# Step 4: Process the Data using PySpark
def process_data_with_spark(data):
    # Define the schema for the data (include new columns)
    schema = ["date", "views", "estimatedMinutesWatched", "subscribersGained", "subscribersLost"]

    # Create the PySpark DataFrame
    df = spark.createDataFrame(data, schema=schema)

    # Example transformation: Cast numeric columns and filter rows
    df = df.withColumn("views", col("views").cast("int"))

    # Example transformation: Filter videos with more than 1000 views
    filtered_df = df.filter(df["views"] > 10)

    # Show the filtered records
    filtered_df.show(10)

    return filtered_df


# Fetch the data and process it
data = fetch_youtube_data()
if data:
    process_data_with_spark(data)
else:
    print("No data found.")
    
def load_data_to_azure_sql(df):
    # Loop over the rows in the PySpark DataFrame and insert into Azure SQL DB
    for row in df.collect():
        insert_query = '''
        INSERT INTO YouTubeAnalytics (date, views, estimatedMinutesWatched, subscribersGained, subscribersLost)
        VALUES (?, ?, ?, ?, ?)
        '''
        cursor.execute(insert_query, row.date, row.views, row.estimatedMinutesWatched, row.subscribersGained, row.subscribersLost)
    conn.commit()
    print("Data loaded into Azure SQL DB successfully.")

# Load transformed data to Azure SQL DB
transformed_df = process_data_with_spark(data)
load_data_to_azure_sql(transformed_df)

# Close connection
cursor.close()
conn.close()

+----------+-----+-----------------------+-----------------+---------------+---+---+
|      date|views|estimatedMinutesWatched|subscribersGained|subscribersLost| _6| _7|
+----------+-----+-----------------------+-----------------+---------------+---+---+
|2023-01-03|   17|                     18|                0|              0|  0|  0|
|2023-01-04|   19|                     30|                0|              0|  0|  0|
|2023-01-06|   30|                     33|                0|              0|  0|  0|
|2023-01-07|   13|                     10|                0|              0|  0|  0|
|2023-01-09|   14|                     11|                0|              0|  0|  0|
|2023-01-10|   21|                     13|                0|              0|  1|  0|
|2023-01-12|   12|                      8|                0|              0|  0|  0|
|2023-01-15|   11|                     12|                0|              0|  1|  0|
|2023-01-16|   19|                     22|                0|     